<a href="https://colab.research.google.com/github/Pamela120609/barchart-udt/blob/master/Projeto_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# -*- coding: utf-8 -*-

from datetime import datetime, timedelta
import google.colab.userdata
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# --- Nosso "caderno" digital: guarda seus itens ---
meus_itens = []


# AGENTE: Gerenciador de Itens (Adicionar)
def adicionar_item(descricao, data, hora, categoria="Pessoal", tipo_item="Evento"):
    """Adiciona um item à lista 'meus_itens'."""
    novo_item = {
        "descricao": descricao,
        "data": data,
        "hora": hora,
        "categoria": categoria,
        "tipo_item": tipo_item
    }
    meus_itens.append(novo_item)
    print(f"✨ Feito!\nAdicionei o {tipo_item} '{descricao}' para você.\nPode deixar comigo! 😉")

# AGENTE: Gerenciador de Itens (Listar)
def listar_itens():
    """Mostra todos os itens da lista, organizados por data e hora."""
    if not meus_itens:
        print("📅 Olhei no caderno e ele está vazio.\nQue tranquilidade! 😊")
    else:
        print("--- Seus próximos itens: ---")

        try:
            meus_itens.sort(key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))
        except Exception as e:
            print(f"Aviso: Não consegui organizar seus itens direito!\nErro: {e} 😔")

        # Mostra cada item
        for i, item in enumerate(meus_itens):
            desc = item.get('descricao', 'Sem descrição')
            data = item.get('data', 'Sem data')
            hora = item.get('hora', 'Sem hora')
            cat = item.get('categoria', 'Sem categoria')
            tipo = item.get('tipo_item', 'Item')
            print(f"{i+1}. [{tipo}] Descrição: {desc}\n   Data: {data}, Hora: {hora}, Categoria: {cat}")
        print("--------------------------")

# --- Conexão com a Inteligência Artificial (Google Gemini) ---


!pip install -q -U google-generativeai

import google.generativeai as genai

chave_api = None
modelo_ia = None

# Tenta conectar e configurar o modelo
try:
    chave_api = google.colab.userdata.get('GOOGLE_API_KEY')
    if not chave_api:
        raise ValueError("Chave de API não encontrada.")

    genai.configure(api_key=chave_api)
    print("Conectado com a IA! ✨\nProntos para conversar! 😉")

    # Carrega o modelo do Gemini que vamos usar
    nome_modelo = 'gemini-2.0-flash'
    modelo_ia = genai.GenerativeModel(nome_modelo)
    print(f"Modelo '{nome_modelo}' carregado. 🎉\nVamos lá!")
    ia_conectada = True

except Exception as e:
    ia_conectada = False
    print(f"ERRO crítico ao conectar com a IA: {e}")
    print("Verifique sua chave 'GOOGLE_API_KEY' nos Secrets do Colab.")



# AGENTE: Ouvinte e Direcionador (Router)

def entender_comando(texto_usuario):
    """Usa a IA pra analisar o texto e descobrir a intenção e detalhes."""
    if modelo_ia is None:
        return "erro_ia", {}


    prompt = f"""
Você é meu assistente pessoal amigável e útil. Analise a frase do usuário e identifique a INTENÇÃO principal e extraia DETALHES relevantes.

Data atual: {datetime.now().strftime('%Y-%m-%d')}. Use pra entender "hoje", "amanhã", etc.

As INTENÇÕES possíveis são:
- adicionar_item: adicionar item (evento, tarefa, aniversário)
- listar_itens: ver itens
- sugerir_pausa: pedir pausa (baseado nos itens)
- analisar_itens: resumo/análise dos itens
- sugerir_nome_criativo: pedir nomes criativos pra um item. Extraia a descrição original no detalhe "descricao_original".
- sugerir_bem_estar_humor: pedir sugestões (atividades, filmes, livros, comidas) baseado no humor
- sugerir_horario_atividade: pedir sugestão de horário pra fazer *qualquer* atividade.
- desconhecido: não entendeu

Sua resposta DEVE seguir este formato simples de texto, uma linha para a intenção e uma linha para cada detalhe:
INTENCAO: [a intenção identificada]
DETALHE: [nome do detalhe]: [valor do detalhe]
DETALHE: [outro detalhe]: [valor]
...

Se não houver detalhes para uma intenção, apenas coloque a linha da INTENCAO.

Exemplos:
Usuário: Adicionar reunião amanhã às 10h.
Resposta:
INTENCAO: adicionar_item
DETALHE: descricao: reunião
DETALHE: data:2024-MM-DD (data de amanhã)
DETALHE: hora: 10:00
DETALHE: tipo_item: Evento

Usuário: Listar meus itens.
Resposta:
INTENCAO: listar_itens

Usuário: Estou cansado, sugira um filme.
Resposta:
INTENCAO: sugerir_bem_estar_humor
DETALHE: humor: cansado
DETALHE: tipo_sugestao: filme

Usuário: Sugira horário para estudar.
Resposta:
INTENCAO: sugerir_horario_atividade

Usuário: Sugira nomes criativos para reunião chata.
Resposta:
INTENCAO: sugerir_nome_criativo
DETALHE: descricao_original: reunião chata

Usuário: {texto_usuario}

Resposta (formato de texto simples):
"""
    try:
        resposta = modelo_ia.generate_content(prompt, generation_config=genai.GenerationConfig(temperature=0.0))
        texto_resposta = resposta.text.strip()


        intent = "desconhecido"
        details = {}

        linhas = texto_resposta.split('\n')
        for linha in linhas:
            if linha.startswith("INTENCAO: "):
                intent = linha.replace("INTENCAO: ", "").strip()
            elif linha.startswith("DETALHE: "):
                partes_detalhe = linha.replace("DETALHE: ", "").split(':', 1)
                if len(partes_detalhe) == 2:
                    nome_detalhe = partes_detalhe[0].strip()
                    valor_detalhe = partes_detalhe[1].strip()
                    details[nome_detalhe] = valor_detalhe

        return intent, details

    except Exception as e:
        print(f"Erro na IA ao entender comando: {e} ")
        return "desconhecido", {}

# AGENTE: Analista Criativo

def analisar_itens_criativo(lista_itens, periodo="próximos itens"):
    """Usa a IA pra analisar itens e gerar resumo criativo."""
    if not lista_itens:
        print(f"Sua lista está vazia.\nNada pra analisar {periodo}. Adicione algo! 😊")
        return
    if modelo_ia is None:
         print("IA não conectada. 😔")
         return

    # Prepara lista pra IA (próximos 10 itens)
    itens_ordenados = sorted(lista_itens, key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))
    hoje = datetime.now().date()
    itens_relevantes = [item for item in itens_ordenados if item.get('data') and datetime.strptime(item['data'], '%Y-%m-%d').date() >= hoje][:10]

    if not itens_relevantes:
         print(f"Sem itens futuros pra analisar {periodo}.\n😊 Tudo livre!")
         return

    lista_formatada = "Próximos itens:\n" + "\n".join([f"- [{i.get('tipo_item', 'Item')}] {i.get('data', '')} {i.get('hora', '')}: {i.get('descricao', 'Sem descrição')}" for i in itens_relevantes])

    # Prompt pra IA analisar
    prompt_analise = f"""
Você é um assistente pessoal amigável e criativo. Analise esta lista de itens:
{lista_formatada}
Gere um resumo criativo ou observação sobre o período. Use emojis, tom humano, quebras de linha. Sugira algo proativo (relaxar, focar).
"""
    try:
        resposta = modelo_ia.generate_content(prompt_analise, generation_config=genai.GenerationConfig(temperature=0.8))
        print(f"\n--- Análise dos seus itens ({periodo}) ---")

        display(to_markdown(resposta.text))
        print("----------------------------------------")
    except Exception as e:
        print(f"Erro ao analisar itens: {e} 😔")

# AGENTE: Sugerir Pausa (usa o Analista Criativo)
def sugerir_pausa_inteligente(lista_itens):
    """Sugere pausas usando a análise criativa dos itens."""
    analisar_itens_criativo(lista_itens, periodo="para sugerir pausas")

# AGENTE: Criador de Nomes
def sugerir_nome_criativo(descricao_original):
    """Usa a IA pra sugerir nomes criativos."""
    if modelo_ia is None:
        print("IA não conectada. 😔")
        return

    # Prompt pra IA sugerir nomes
    prompt_sugestao_nome = f"""
Você é um assistente pessoal divertido. Sugira 3-5 nomes criativos/divertidos para "{descricao_original}". Use emojis, tom humano, liste com quebras de linha.
"""
    try:
        resposta = modelo_ia.generate_content(prompt_sugestao_nome, generation_config=genai.GenerationConfig(temperature=0.9))
        print(f"\n--- Nomes divertidos para '{descricao_original}' 🤔 ---")
        display(to_markdown(resposta.text))
        print("----------------------------------------------------")
    except Exception as e:
        print(f"Erro ao sugerir nomes: {e} 😔")

# AGENTE: Bem-Estar Personalizado
def sugerir_bem_estar_com_humor(humor, tipo_sugestao):
    """Usa a IA pra sugerir bem-estar baseado no humor e tipo."""
    if modelo_ia is None:
        print("IA não conectada. 😔")
        return

    # Prompt pra IA sugerir bem-estar
    prompt_bem_estar = f"""
Você é um assistente pessoal empático e com humor. Usuário está "{humor}", quer sugestões de "{tipo_sugestao}". Sugira 3-5 coisas (atividades, filmes, livros, comidas) que combinem. Use emojis, tom humano, liste com quebras de linha.
"""
    try:
        resposta = modelo_ia.generate_content(prompt_bem_estar, generation_config=genai.GenerationConfig(temperature=0.9))
        print(f"\n--- Algo bom pra você, já que está {humor}... 😊 ---")
        display(to_markdown(resposta.text))
        print("---------------------------------------------------")
    except Exception as e:
        print(f"Erro ao sugerir bem-estar: {e} 😔")

# AGENTE: Produtividade e Planejamento
def sugerir_horario_atividade(lista_itens):
    """Orquestra a sugestão de horário para qualquer atividade."""
    if modelo_ia is None:
        print("IA não conectada. 😔")
        return

    print("\nOk, vamos encontrar o melhor momento para você fazer uma atividade! ✨")


    tipo_atividade = input("Que atividade você quer agendar? (Ex: Estudar Python, Malhar, Ler um livro): ").strip()

    if tipo_atividade.lower() == 'sair do chat':
        return

    periodo_produtivo = input("Qual período do dia você se sente mais produtivo para isso? (Manhã, Tarde, Noite): ").strip()

    if periodo_produtivo.lower() == 'sair do chat':
        return


    if not tipo_atividade or not periodo_produtivo:
        print("Preciso saber a atividade e seu período produtivo pra te ajudar melhor. 😉")
        return

    # Prepara a lista de itens para a IA
    itens_ordenados = sorted(lista_itens, key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))
    hoje = datetime.now().date()
    itens_relevantes = [item for item in itens_ordenados if item.get('data') and datetime.strptime(item['data'], '%Y-%m-%d').date() >= hoje and datetime.strptime(item['data'], '%Y-%m-%d').date() <= (hoje + timedelta(days=7))]

    lista_formatada = "Próximos itens (7 dias):\n" + ("Nenhum." if not itens_relevantes else "\n".join([f"- [{i.get('tipo_item', 'Item')}] {i.get('data', '')} {i.get('hora', '')}: {i.get('descricao', 'Sem descrição')}" for i in itens_relevantes]))

    # Prompt pra IA sugerir horário para a atividade geral em formato de texto simples
    prompt_sugestao = f"""
Você é um assistente pessoal inteligente. Analise esta lista de itens:
{lista_formatada}
Usuário quer agendar a atividade "**{tipo_atividade}**".
Ele é mais produtivo no período da "**{periodo_produtivo}**".
Sugira o melhor dia/horário nos próximos 7 dias, dando preferência ao período produtivo e procurando por blocos livres.
Responda neste formato de texto simples:
SUGESTAO_DATA: [data sugerida YYYY-MM-DD]
SUGESTAO_HORA: [hora sugerida HH:MM]
MENSAGEM: [Mensagem amigável com quebras de linha sugerindo a atividade]
"""
    try:
        print("Analisando seus itens e preferências... 🕵️‍♂️")
        resposta = modelo_ia.generate_content(prompt_sugestao, generation_config=genai.GenerationConfig(temperature=0.5))
        texto_resposta = resposta.text.strip()


        data_sug = None
        hora_sug = None
        msg_sug = "Encontrei um possível horário para você fazer essa atividade."

        linhas_resposta_prod = texto_resposta.split('\n')
        for linha in linhas_resposta_prod:
            if linha.startswith("SUGESTAO_DATA: "):
                data_sug = linha.replace("SUGESTAO_DATA: ", "").strip()
            elif linha.startswith("SUGESTAO_HORA: "):
                hora_sug = linha.replace("SUGESTAO_HORA: ", "").strip()
            elif linha.startswith("MENSAGEM: "):
                msg_sug = linha.replace("MENSAGEM: ", "").strip()


        if data_sug and hora_sug:
            print(f"\n--- Sugestão de Horário para {tipo_atividade} ---")
            display(to_markdown(msg_sug))
            print(f"Sugestão: Dia {data_sug} às {hora_sug}")
            print("--------------------------------------------")


            confirmar = input(f"Quer adicionar '{tipo_atividade}' na sua lista? (sim/não): ").strip().lower()

            if confirmar.lower() == 'sair do chat':
                return


            if confirmar == 'sim':

                # AGENTE GERENCIADOR DE ITENS (Adicionar)
                adicionar_item(tipo_atividade, data_sug, hora_sug, categoria="Produtividade", tipo_item="Atividade")
                print(f"'{tipo_atividade}' adicionada! 💪")
            elif confirmar in ['não', 'nao']:
                print("Ok! 😉")

                tentar = input(f"Tentar achar outro horário para '{tipo_atividade}'? (sim/não): ").strip().lower()

                if tentar.lower() == 'sair do chat':
                    return

                if tentar == 'sim':

                    sugerir_horario_atividade(meus_itens)
                else:
                    print("Tudo bem! Me diga se precisar de mais alguma coisa. 😊")
            else:
                print("Não entendi. Deixo a sugestão anotada! 😊")
        else:
             print("Não consegui achar um horário ideal. 😔 Sua lista tá cheia ou as preferências são restritivas?")

    except Exception as e: # Captura qualquer erro no processamento da resposta da IA
        print(f"Erro ao processar sugestão de horário: {e} 😭")


# --- O ORQUESTRADOR Principal ---

def iniciar_chatbot():
    """Inicia o assistente e orquestra os agentes."""
    print("✨ Oi! Eu sou seu Assistente Pessoal IA! ✨📅🧘‍♀️🧠")
    print("Posso organizar seus itens e dar umas dicas legais!")
    print("Eu uso IA e o conceito de AGENTES (funções especializadas) pra te ajudar.")
    print("\nO que posso fazer? 🤔")
    print("- 'Adicionar [item]' (Ex: 'Adicionar reunião amanhã 10h', 'tarefa comprar pão hoje', 'aniversário João 15 dez')")
    print("- 'Listar meus itens'")
    print("- 'Sugira uma pausa'")
    print("- 'Analisar meus itens [período]' (Ex: 'hoje', 'esta semana')")
    print("- 'Sugira nomes criativos para [descrição]'")
    print("- 'Estou [humor], sugira [tipo]' (Ex: 'cansado, sugira filme')")
    print("- 'Sugira horário para [atividade]' (Ex: 'Sugira horário para estudar', 'Sugira horário para malhar')")
    print("\nDigite o que precisa! 😊")
    print("**Escreva 'sair do chat' para terminar.** 👋")
    print("-" * 60)

    # Verifica se a IA tá pronta
    if not ia_conectada:
        # Lista as funções que NÃO vão funcionar sem a IA
        print("\nOps! IA não conectada. 😟 Algumas funções (analisar, sugerir pausa, sugerir nome, sugerir bem-estar, sugerir horário) não vão funcionar.")


    while True:
        comando_usuario = input("Você: ")

        # --- VERIFICAÇÃO IMEDIATA PARA SAIR ---
        if comando_usuario.lower() == 'sair do chat':
            print("Até mais! Cuide-se! 😉👋")
            break


        print("Carregando...")
        intent, details = entender_comando(comando_usuario)


        print(f"\nDEBUG: Intenção identificada pelo Ouvinte: {intent}")
        print(f"DEBUG: Detalhes extraídos pelo Ouvinte: {details}")
        print("-" * 20)

        # --- O ORQUESTRADOR decide qual AGENTE (função) chamar ---

        if intent == "adicionar_item":
            print("Ok, vou adicionar isso.")

            desc = details.get('descricao', 'Novo Item')
            data = details.get('data', '')
            hora = details.get('hora', '')
            cat = details.get('categoria', 'Pessoal')
            tipo = details.get('tipo_item', 'Evento')
            if desc and data and data != "Sem data":
                 if tipo == "Evento" and not hora:
                     print("Pra Evento preciso da hora também. 😅 Poderia especificar? (Ex: 'Adicionar evento reunião amanhã as 10:00')")
                 else:

                     adicionar_item(desc, data, hora, cat, tipo)

            else:
                 print("Ops! Não consegui pegar a **descrição e a data** do seu pedido. 🙁 Tente de novo? (Ex: 'Adicionar tarefa comprar pão hoje' ou 'Adicionar aniversário do João dia 15 de dezembro')")


        elif intent == "listar_itens":
            print("Claro! Olhando seus itens...")
            listar_itens()

        elif intent == "sugerir_pausa":
            if not ia_conectada:
                 print("Não consigo sugerir pausas agora, a IA não está conectada. 😔")
                 continue

            print("Pensando num break? Boa! 🧘‍♀️")
            sugerir_pausa_inteligente(meus_itens)


        elif intent == "analisar_itens":
            if not ia_conectada:
                 print("Não consigo analisar seus itens agora, a IA não está conectada. 😔")
                 continue


            periodo = details.get('periodo', 'próximos itens')
            print(f"Analizando seus itens para {periodo}... 🕵️‍♀️")
            analisar_itens_criativo(meus_itens, periodo=periodo)


        elif intent == "sugerir_nome_criativo":
             if not ia_conectada:
                 print("Não consigo sugerir nomes agora, a IA não está conectada. 😔")
                 continue


             desc_original = details.get('descricao_original', '')

             # --- Lógica Corrigida para Sugerir Nomes ---
             if not desc_original:

                 print("Qual item você quer deixar mais divertido? 🤔 Diga a descrição atual:")
                 desc_original = input("Você: ").strip()
                 if desc_original.lower() == 'sair do chat':
                     continue

             if desc_original:
                 print(f"Renomear '{desc_original}'? Adoro um desafio criativo! ✨")
                 sugerir_nome_criativo(desc_original)
             else:
                 print("Hmm, não consegui pegar a descrição do item. Tente de novo? 😉")



        elif intent == "sugerir_bem_estar_humor":
             if not ia_conectada:
                 print("Não consigo sugerir atividades agora, a IA não está conectada. 😔")
                 continue


             humor = details.get('humor', '')
             tipo = details.get('tipo_sugestao', '')
             if humor and tipo:
                 print(f"Entendido! {humor}, quer sugestões de {tipo}. 🤔")
                 sugerir_bem_estar_com_humor(humor, tipo)
             else:
                 print("Pra te dar sugestões personalizadas, preciso saber como você está se sentindo e o que você quer (atividades, filmes, livros, comidas). 😉 Tente algo como: 'Estou cansado, sugira um filme'.")

        # --- Lógica: Sugerir Horário para Qualquer Atividade ---
        elif intent == "sugerir_horario_atividade":
            if not ia_conectada:
                 print("Não consigo sugerir horários agora, a IA não está conectada. 😔")
                 continue

            # Chama o Agente Produtividade e Planejamento (implementado como função)
            sugerir_horario_atividade(meus_itens)
        # --- Fim da Lógica ---

        # Se a IA nao reconhecer a mensagem
        elif intent == "desconhecido":
            print("Desculpe, não entendi bem o que você disse. 😅 Minha inteligência artificial às vezes precisa de uma ajudinha!")
            print("Posso ajudar com itens (adicionar, listar, analisar), pausas, nomes criativos, bem-estar por humor, ou sugerir horário para uma atividade.")
            print("Tente: 'Adicionar tarefa...', 'Listar meus itens', 'Sugira uma pausa', 'Estou feliz, sugira filme', ou **'Sugira horário para estudar'**, **'Sugira horário para malhar'**. 😉")

        # Se deu algum erro na conexão com a IA
        elif intent == "erro_ia":
             print("Problema com a IA. 😔 Tente de novo mais tarde!")
        else:
            print(f"Hmm, essa intenção '{intent}' é nova pra mim. Ainda estou aprendendo! 🚧")


iniciar_chatbot()


Conectado com a IA! ✨
Prontos para conversar! 😉
Modelo 'gemini-2.0-flash' carregado. 🎉
Vamos lá!
✨ Oi! Eu sou seu Assistente Pessoal IA! ✨📅🧘‍♀️🧠
Posso organizar seus itens e dar umas dicas legais!
Eu uso IA e o conceito de AGENTES (funções especializadas) pra te ajudar.

O que posso fazer? 🤔
- 'Adicionar [item]' (Ex: 'Adicionar reunião amanhã 10h', 'tarefa comprar pão hoje', 'aniversário João 15 dez')
- 'Listar meus itens'
- 'Sugira uma pausa'
- 'Analisar meus itens [período]' (Ex: 'hoje', 'esta semana')
- 'Sugira nomes criativos para [descrição]'
- 'Estou [humor], sugira [tipo]' (Ex: 'cansado, sugira filme')
- 'Sugira horário para [atividade]' (Ex: 'Sugira horário para estudar', 'Sugira horário para malhar')

Digite o que precisa! 😊
**Escreva 'sair do chat' para terminar.** 👋
------------------------------------------------------------
Você: sugira horario
Carregando...

DEBUG: Intenção identificada pelo Ouvinte: sugerir_horario_atividade
DEBUG: Detalhes extraídos pelo Ouvinte: {}
---

> Que ótimo que você está feliz!

Sugestão: Dia 2024-07-06 às 20:00
--------------------------------------------
Quer adicionar 'estou feliz quero ver filme' na sua lista? (sim/não): sim
✨ Feito!
Adicionei o Atividade 'estou feliz quero ver filme' para você.
Pode deixar comigo! 😉
'estou feliz quero ver filme' adicionada! 💪
Você: estou feliz quero ver filme
Carregando...

DEBUG: Intenção identificada pelo Ouvinte: sugerir_bem_estar_humor
DEBUG: Detalhes extraídos pelo Ouvinte: {'humor': 'feliz', 'tipo_sugestao': 'filme'}
--------------------
Entendido! feliz, quer sugestões de filme. 🤔

--- Algo bom pra você, já que está feliz... 😊 ---


> Ebaaa, que bom que você está feliz! 😄 Transbordando alegria assim, dá até vontade de sair dançando! 💃🕺 Já sei, que tal prolongarmos essa vibe boa com um combo de coisas legais? Preparei algumas sugestões que combinam super com esse seu bom humor:
> 
> 1.  **Filme:** "Sing 2" 🎤 - Prepare a pipoca e solte a voz! Essa animação é pura energia positiva, com músicas que grudam e uma história inspiradora sobre acreditar nos seus sonhos. Impossível não sair cantando e sorrindo! 😁🎶
> 
> 2.  **Atividade:** Piquenique no parque 🧺🌳 - Chame os amigos, prepare sanduíches gostosos, leve um frisbee e aproveite o sol! Uma tarde relaxante em contato com a natureza é um ótimo jeito de recarregar as energias e compartilhar essa felicidade com quem você ama. 🌞
> 
> 3.  **Comida:** Brigadeiro gourmet 🍫😋 - Ah, não tem como ser infeliz comendo brigadeiro, né? Experimente fazer uma receita gourmet, com ingredientes especiais e diferentes sabores. Além de delicioso, cozinhar pode ser uma terapia super divertida! 😉
> 
> 4.  **Livro:** "O Pequeno Príncipe" 👑✨ - Uma leitura leve e inspiradora, que te faz refletir sobre a vida, a amizade e a importância de valorizar as pequenas coisas. Perfeito para manter o coração quentinho e a mente aberta para novas aventuras! 🚀
> 
> 5.  **Música:** Monte uma playlist animada! 🎶🎧 - Crie uma seleção de músicas que te façam dançar, cantar e sorrir. Vale misturar hits do momento com aqueles clássicos que sempre te alegram. Depois, é só dar o play e deixar a felicidade tomar conta! 🎉
> 
> Aproveite cada momento e continue irradiando essa alegria por aí! ✨ Se precisar de mais sugestões, é só chamar! 😉


---------------------------------------------------
Você: sair do chat
Até mais! Cuide-se! 😉👋
